# 手撕Transformer

本文件将按照"Attention is all you need"论文中的结构图对涉及到的组件进行逐一实现，架构图如下所示：

![Transformer架构图](resource/transformer_frame.png)

## 位置编码Positional Encoding

因为Transformer架构不同于串行处理的RNN和LSTM，所以需要额外引入位置编码给输入序列注入位置信息。

### 绝对正弦-余弦位置编码（原版编码方式）

位置编码计算公式如下图所示：

![正余弦位置编码公式](resource/sincosPE.png)

公式中的参数解释如下：

1. p表示输入序列分词后某个token所在的位置索引

2. i表示当前所属维度，分奇偶进行成对计算

3. d表示维度，此处的维度与分词得到token后，token经过embedding层所得得到的向量表示的维度（位置编码要与序列编码相加，所以维度要相同）

In [1]:
import torch
import torch.nn as nn
import math


class SincosPE(nn.Module):
    # d_model表示维度，max_seq_length提前创建长度为80的位置编码（固定不改变）
    def __init__(self, d_model, max_seq_length=80):
        super().__init__()
        
        self.d_model = d_model
        # 创建PE矩阵
        pe = torch.zeros(max_seq_length, d_model)
        for pos in range(max_seq_length):
            for i in range(0, d_model, 2):
                # 实现公式对应的位置编码计算
                pe[pos, i] = math.sin(pos / (100000) ** ((2 * i) / d_model))
                pe[pos, i + 1] = math.cos(pos / (100000) ** ((2 * i) / d_model))

        # 此时的PE矩阵是二维的，需要升维满足batch训练的shape
        pe = pe.unsqueeze(0)        # [1, max_seq_length, d_model]

        # 此时的pe在输入序列确定的时候就已经是确认不再发生改变的状态了
        # 参数形式nn.Parameter存储优化器会尝试更新，不符合要求
        # 以类内变量self.pe=pe存储，CPU/GPU切换时会增加手动处理的复杂
        self.register_buffer('pe', pe)      # torch进行自动管理，且不进行梯度计算和更新

    def forward(self, x):
        # 此时传入的x是输入序列经过分词嵌入后得到的
        # 原论文提及需要进行放缩，理由是避免与位置编码相加时数值过小导致位置信号的稀释
        x = x * math.sqrt(self.d_model)     # 根号d是经验数值

        # 因为pe是固定80创建的，避免影响到eos_token等特殊字符，此处只对有效位置进行加和
        seq_len = x.size(1)     # x的shape是[batch, seq_len, d_model]
        x = x + self.pe[:, :seq_len]
        return x

In [ ]:
PE = SincosPE(8)
PE.pe

### 旋转位置编码RoPE

## 多头注意力机制Multihead Attention

此步骤即是对Transformer的核心机制Self-Attention的实现，多头则是处理不同序列跨度的多个独立的注意力头。Self-Attention的一个输出向量计算如下图所示：

![Self-Attention推导示意图](resource/self_attention.png)

Multi-head Attention结构如下图所示：

![Multi-head Attention结构示意图](resource/multi-head.png)

### 重点参数及原因解析

在论文"Attention is all you need"的原文对Multi-head Attention的表述是这样的：

Instead of performing a single attention function with dmodel-dimensional keys, values and queries, we found it beneficial to linearly project the queries, keys and values h times with different, learned linear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of queries, keys and values we then perform the attention function in parallel, yielding dv-dimensional output values. These are concatenated and once again projected, resulting in the final values。

上述话所阐述的关键在于，原本Self-Attention中所有维度的信息集中在一个注意力头回被迫“平均”学习所有类型的关系，导致任何关系都无法学的好。**因此，将原维度平均拆分成h个更小的维度，每个维度单独学习特定类型的特征或关系效果将更好**

**上述描述所需的参数如下**

1. **d_model：** 经过分词及词嵌入层后所产生的维度
2. **d_k：** 每个注意力头所分配的维度
3. **heads：** 注意力头的数量

多头注意力的计算方式也与单头存在不同，如下所示原文计算公式：

![Multi-head Attention计算公式](resource/multi-head_attention.png)

公式中每个头的attention的计算都是原始的$Q, K, V$矩阵乘上对应头的参数矩阵$W_i$，去获取每个头专属的独立空间的$Q_i, K_i, V_i$，再去计算attention score，此时的的$W_i^Q, W_i^K, W_i^V$则是可学习的将原始输入映射至专属空间的**投影矩阵**。

根据线性代数分块矩阵的原理 **(证明过程可见项目内的PDF学习笔记)**，因为每个注意力头都是独立，不依赖于前置注意力头的计算结果，所以可以将$Q, K, V$分别的权重矩阵$W$进行合并如下：

合并前：$W_i^Q = W_i^K = W_i^V = (d_{model}, d_k)$

合并操作：$W^Q = Concat[W_1^Q, W_2^Q, ..., W_h^Q]$

合并后：$W^Q = (d_{model}, d_{model})$

得到的QKV:(batch_size, seq_len, h, d_model)，先一次性得到完整的QKV，再进行分头

**数据shape变换过程如下：**

1. 输入序列经过分词后：**(batch_size, seq_len)**
2. 经过embedding层嵌入后的输入数据x：**(batch_size, seq_len, d_model)**
3. 分头后注意力头数为h的QKV：**(batch_size, seq_len, h, d_model/h)**

获取矩阵$Q, K, V$的最终目的都是为了计算attention score去学习序列信息，过程中涉及到矩阵乘法，关键点之一在于，**@/mutual只会对matrix的最后两个dimensions进行矩阵计算**，其他的dimensions视为batch，而我们真正要参与计算的维度通过self-attention可以知道，是seq_len和d_k，所以进行维度变换：

**(batch_size, seq_len, h, d_k) ---transpose--> (batch_size, h, seq_len, d_k)**

如果不进行维度变换直接点积，那么其含义则对不同注意力头的信息进行相似度计算，而真正所需要的是在同一个头的内部，计算一个词与其他词的相似度

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class MultiHeadAttention(nn.Module):

    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        # 参考上文权重矩阵合并的shape进行理解
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

        # 最后的线性层，连接所有注意力头的输出信息
        self.out = nn.Linear(d_model, d_model)
    
    # 此处的attention是基于分头后的shape进行计算
    def attention(self, q, k, v, d_k, mask=None, dropout=None):
        # 注意力分数，k需要转置
        # 注意此处的根号d，不同于self attention中的值
        scores = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)      
        
        if mask is not None:
            # scores的shape应该是(batch, h, seq_len, d_k)
            # mask的原始shape是(batch, seq_len, d_k)
            # 所以需要在dim=1的位置上进行升维，确保mask覆盖位置正确
            mask = mask.unsqueeze(1)
            # 0表示被屏蔽，用-inf进行代替
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 此处的softmax应该是对一个token关于其他所有token的匹配分数进行归一化
        # (batch, h, seq_len, d_k)则是对d_k进行softmax
        scores = F.softmax(scores, dim=-1)

        if dropout is not None:
            scores = dropout(scores)
        
        output = torch.matmul(scores, v)        # 此处不要进行额外转置
        return output
    
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)      # 获取batch_size

        # 投影及多头拆分
        # 对k进行映射，此时最开始传入的k可以看作是x或是上一层输出
        # 即q=k=v=x，映射完后即为广义上的Key
        k = self.k_linear(k)        # (batch, seq_len, d_model)
        # 将原始k的(seq_len, d_mdoel)升为三维，最后两维的shape为(self.h, self.d_k)，即分头操作
        k = k.view(bs, -1, self.h, self.d_k)        # (batch, seq_len, head, d_k)    
        k = k.transpose(1,2)        # (batch, head, seq_len, d_k)

        q = self.q_linear(q).view(bs, -1, self.h, self.d_k).transpose(1, 2)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k).transpose(1, 2)

        scores = self.attention(q, k, v, self.d_k, mask, self.dropout)      # (batch, head, seq_len, d_k)

        # 进行多头合并
        # contiguous确保tensor在内存地址连续，保证view的成功reshape
        # 原因：transpose只改变索引映射，没有重新排布实际内存
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)         # (batch, seq_len, d_model)

        out = self.out(concat)
        return out

d:\miniconda\envs\transformer\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
